In [ ]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
TSUN = 4.9169e-6
gbfisher_columns = ['f','colatitude','longitude','amplitude','inclination','polarization','phase','fdot','fddot','sigma_df/f','sigma_colatitude','sigma_longitude','sigma_dA/A','sigma_inclination','sigma_polarization','sigma_phase','sigma_dfdot/fdot','sigma_dfddot/fddot','sigma_Omega','SNR','b','l','mchirp','distance'] 
gbfisher = pd.read_table('../../ldasoft/gbfisher_example/gbfisher_parameters.dat',delimiter=' ', header=None,index_col=False,names=gbfisher_columns)
gbfisher['inclination'] = 90 - abs(np.degrees(gbfisher['inclination']) - 90)
gbfisher['sigma_inclination'] = 90 - abs(np.degrees(gbfisher['sigma_inclination']) - 90)
gbfisher['sigma_mchirp'] = (1/(5*TSUN))*((5/3)**(3/5))*(np.pi**(-8/5))*(gbfisher['f']**(-11/5))*(gbfisher['fdot']**(3/5))*((121*(gbfisher['sigma_df/f']**2) + 9*(gbfisher['sigma_dfdot/fdot']**2))**(1/2))
gbfisher.head()

In [ ]:
def readData(raw, index, inFunc=None):
    x = []
    for pos, line in enumerate(raw):
        values = [float(x) for x in line.split()]
        if inFunc==None:
            x.append(values[index])
        else:
            x.append(inFunc(values[index]))
    return x

In [ ]:
p = 'out'
exclude = ['fdot','inc']
binDir = os.path.join(os.getcwd(),f'../{p}-gwprior')

for binary in os.listdir(binDir):
    rawData = []
    for folder in os.listdir(os.path.join(binDir,binary)):
        if os.path.isdir(os.path.join(binDir,os.path.join(binary,folder))) & (not folder in exclude):
            B_n = int(re.search('binary(.+?)row',folder).group(1))
            with open(os.path.join(binDir,os.path.join(binary,os.path.join(folder,'post_equal_weights.dat'))),'r') as raw:
                rawData = np.concatenate([rawData,readData(raw,3,inFunc=lambda p: 24*60*p)])
    if B_n == 1:
        vdata = pd.DataFrame(np.sort(rawData))
        vdata.insert(0, 'B_n', B_n)
        vdata.insert(0, 'type', 'gw+em')
    else:
        raw_vdata = pd.DataFrame(np.sort(rawData))
        raw_vdata.insert(0, 'B_n', B_n)
        raw_vdata.insert(0, 'type', 'gw+em')
        vdata = pd.concat([vdata,raw_vdata], axis=0)

In [ ]:
chainDir = '../../data/08yr_sp32_100_binaries/'
trvs = []

for binary in os.listdir(chainDir):
    rawData = []
    B_n = int(re.search('binary(.*)',binary).group(1))
    with open(os.path.join(chainDir,os.path.join(binary,'chains/dimension_chain.dat.1')),'r') as raw:
        rawData = np.concatenate([rawData,readData(raw,0,inFunc=lambda f: 2/(60*f))])
    raw_vdata = pd.DataFrame(np.sort(rawData))
    raw_vdata.insert(0, 'B_n', B_n)
    raw_vdata.insert(0, 'type', 'gw')
    vdata = pd.concat([vdata,raw_vdata], axis=0) 
    if (B_n%20==0):
        print(f'{str(B_n)}%')

In [ ]:
trvs = list(2/(60*np.asarray(gbfisher['f'])))
vdata = vdata.rename(columns={0:'value'})
vdata.insert(3, 'true_value', 0)
vdata['true_value'] = np.asarray(trvs)[vdata['B_n']-1]
vdata = vdata.sort_values(['true_value','type','value'])
vdata['value'] = 60*(10^6)*(vdata['value'] - vdata['true_value'])
vdata['B_n'] = vdata['B_n'].map(dict(zip(vdata['B_n'].unique(),np.arange(1,len(vdata['B_n'].unique())+1))))

In [ ]:
b_raw = np.array([np.unique(c).tolist() for c in [vdata.loc[(vdata['B_n'] == b), 'B_n'] for b in vdata['B_n'].unique()]])
b_n = np.unique(b_raw[b_raw.astype(bool)].tolist()).tolist()
gw = [vdata.loc[(vdata['B_n'] == b) & (vdata['type'] == 'gw'),'value'] for b in b_n]
gwem = [vdata.loc[(vdata['B_n'] == b) & (vdata['type'] == 'gw+em'), 'value'] for b in b_n]
tv = vdata['true_value'].unique()

In [ ]:
cut = [0,1]
qgw = [[0.0025,0.9975] for i in range(len(cut))]
qgwem = [[0.0025,0.9975] for i in range(len(cut))]
wt = 4
#######################################################
fig = plt.figure(figsize=(36,18))
ax = fig.add_subplot(111)
ctv = tv[cut]

sgw = [pd.Series(l) for l in (np.asarray([s.tolist() for s in gw])[cut])]
cgw = [list(l) for l in [s[s.between(s.quantile(qgw[i][0]),s.quantile(qgw[i][1]))] for i,s in enumerate(sgw)]]

vp1 = plt.violinplot(cgw, positions=ctv, widths=wt, showextrema=False)
for pc in vp1['bodies']:
    pc.set_facecolor('#226EF1')
    pc.set_edgecolor('black')
    pc.set_alpha(0.5)
    pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0],-np.inf,np.mean(pc.get_paths()[0].vertices[:, 0]))

mgw = [np.percentile(list(m),50) for m in sgw]
hgw = [np.percentile(list(m),90) for m in sgw]
lgw = [np.percentile(list(m),10) for m in sgw]
plt.hlines(mgw,ctv-wt/2,ctv,linewidth=2.0)
plt.hlines(lgw,ctv-0.89*wt/4,ctv,linewidth=1.0,linestyle='--')
plt.hlines(hgw,ctv-0.89*wt/4,ctv,linewidth=1.0,linestyle='--')

sgwem = [pd.Series(l) for l in (np.asarray([s.tolist() for s in gwem])[cut])]
cgwem = [list(l) for l in [s[s.between(s.quantile(qgwem[i][0]),s.quantile(qgwem[i][1]))] for i,s in enumerate(sgwem)]]

vp2 = plt.violinplot(cgwem, positions=ctv, widths=wt, showextrema=False)
for pc in vp2['bodies']:
    pc.set_facecolor('#E67E00')
    pc.set_edgecolor('black')
    pc.set_alpha(0.5)
    pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0],np.mean(pc.get_paths()[0].vertices[:, 0]),np.inf)

mgwem = [np.percentile(list(m),50) for m in sgwem]
hgwem = [np.percentile(list(m),90) for m in sgwem]
lgwem = [np.percentile(list(m),10) for m in sgwem]
plt.hlines(mgwem,ctv+wt/2,ctv,linewidth=2.0)
plt.hlines(lgwem,ctv+0.89*wt/4,ctv,linewidth=1.0,linestyle='--')
plt.hlines(hgwem,ctv+0.89*wt/4,ctv,linewidth=1.0,linestyle='--')

plt.legend([vp1['bodies'][0],vp2['bodies'][0]],['GW Posterior','EM Posterior'],loc='upper right',prop={'size': 24})
plt.xlabel('True Period (minutes)',fontsize=30)
plt.ylabel('Residual Period ($\mu$s)',fontsize=30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.grid()